In [2]:
import h5py
import numpy as np
import pandas as pd 

import fastdtw
import scipy.spatial.distance as sd

from scipy import ndimage 
from scipy.cluster import hierarchy 
from scipy.spatial import distance_matrix 
from sklearn import manifold, datasets 
from sklearn.cluster import AgglomerativeClustering

import matplotlib.pyplot as plt
import seaborn as sns 
plt.rcParams["figure.figsize"] = [16,9]
sns.set_style('whitegrid')

GRID_SIZE = 0.06
LOWER_LAMBDA = 6472.5
UPPER_LAMBDA = 6740
PLOT_MASK = range(1475,1550)

In [3]:
hf_spec = h5py.File("/data/praveen/GDR3_non_null_resampled_spectra.h5", "r")
df1 = pd.DataFrame(hf_spec["spectra"])

hf_spec_2 = h5py.File("/data/praveen/GDR3_missed60K_resampled_spectra.h5", "r")
df2 = pd.DataFrame(hf_spec_2["spectra"])

frames = [df1, df2]
result = pd.concat(frames)

In [4]:
df_masked = result.loc[:,PLOT_MASK]
df_masked_np = df_masked.to_numpy()

In [5]:
df_masked_np

array([[1.00292947, 1.02009993, 1.02449091, ..., 1.01727503, 1.0032992 ,
        0.99269212],
       [0.91397414, 0.92508009, 0.91401545, ..., 0.97527804, 0.96492472,
        0.94062854],
       [0.85256761, 0.85561886, 0.85255965, ..., 0.88472869, 0.87888464,
        0.85965847],
       ...,
       [0.75763265, 0.76384388, 0.75887171, ..., 0.80009313, 0.79944416,
        0.81120826],
       [0.89527092, 0.89327475, 0.89152492, ..., 0.9169642 , 0.92062587,
        0.91718901],
       [0.76812834, 0.76685381, 0.77277509, ..., 0.80705675, 0.81220549,
        0.81335903]])

In [6]:
def my_fastdtw(spec1, spec2):
    return fastdtw.fastdtw(spec1, spec2)[0]

In [7]:
distance_matrix = sd.pdist(df_masked_np, my_fastdtw)

MemoryError: Unable to allocate 1.26 TiB for an array with shape (173074036996,) and data type float64

In [ ]:
distance_matrix_square_form = sd.squareform(distance_matrix)

In [ ]:
hf_dtwdistance = h5py.File("/data/praveen/dtw_distances_all_galah.h5","w")

In [ ]:
hf_dtwdistance.create_dataset("dtw_distance", data=distance_matrix_square_form)
hf_dtwdistance.close()

In [ ]:
def distance_cost_plot(distances):
    im = plt.imshow(distances, interpolation='nearest', cmap='Blues') 
    plt.gca().invert_yaxis()
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.grid()
    plt.colorbar();

In [ ]:
distance_cost_plot(distance_matrix_square_form)